# **Installing Requirements**

Since the project is developed by different people, we will install all the requirements using the requirements.txt file which specifies all the packets' version that must be installed.

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


# **Downloading Files from GDrive**

# **Face Recognition**

Face recognition is a computer vision task that involves identifying and verifying a person's identity based on their facial features. This process can be broken down into these steps:

1. **Detection**: Identifying faces in images or video frames.
2. **Feature** Extraction: Capturing unique facial characteristics.
3. **Representation**: Creating a distinctive template for each face.
4. **Model Training**: Associating templates with known identities during training.
5. **Matching**: Comparing a new face's template to stored ones for identification.
6. **Decision**: Determining a match based on a similarity threshold.

Nowadays, these steps are performed through deep learning models. In the following section we will provide a simple implementation through a pre-trained model and our paper implementation (further details in the next sections).

## **Load all the user faces**

All the registered users' face are saved into the UserFaces folder. We have to build our "dataset" of faces on which the recognition will be performed.

In [11]:
import os
import cv2
import dlib
import numpy as np

def face_rects(image, face_detector):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale image
    rects = face_detector(gray, 1)
    # return the bounding boxes
    return rects


def face_landmarks(image, shape_predictor, face_detector):
    return [shape_predictor(image, face_rect) for face_rect in face_rects(image, face_detector)]


def face_encodings(image, face_encoder, shape_predictor, face_detector):
    # compute the facial embeddings for each face 
    # in the input image. the `compute_face_descriptor` 
    # function returns a 128-d vector that describes the face in an image
    return [np.array(face_encoder.compute_face_descriptor(image, face_landmark)) 
            for face_landmark in face_landmarks(image, shape_predictor, face_detector)]


face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("../Models/shape_predictor_68_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("../Models/dlib_face_recognition_resnet_model_v1.dat")

# Initialize lists to store known faces and their corresponding names
known_faces = {}

base_directory = "../UserFaces/" # Directory containing user faces

# Iterate through directories
for user_name in os.listdir(base_directory):

    user_path = os.path.join(base_directory, user_name)

    # Iterate through face images in each user directory
    for filename in os.listdir(user_path):
        image_path = os.path.join(user_path, filename)

        img = cv2.imread(image_path) # Read the image
        new_encodings = face_encodings(img, face_encoder, shape_predictor, face_detector)

        encodings = known_faces.get(user_name, [])
        encodings.extend(new_encodings)
        known_faces[user_name] = encodings


print(known_faces.keys())

dict_keys(['Daniele'])
{'Daniele': [array([-0.02986787, -0.04006093,  0.00694538, -0.04759768, -0.10049356,
        0.02562953, -0.01679501, -0.08864886,  0.18726619, -0.03456683,
        0.18501316, -0.03232293, -0.19114077,  0.0444179 , -0.02776724,
        0.05532715, -0.05235158, -0.101264  , -0.11612409, -0.11691881,
       -0.10209946, -0.03569508, -0.04508139,  0.09246112, -0.19649988,
       -0.17695811, -0.12603174, -0.17035078,  0.02675712, -0.12949145,
       -0.11669026,  0.05573684, -0.13280265, -0.10324054,  0.07697368,
        0.0656424 , -0.07010015, -0.08588725,  0.21078998,  0.03849274,
       -0.105121  ,  0.04550758,  0.09148104,  0.27920941,  0.16162728,
        0.04018471,  0.00549568, -0.05843762,  0.17402168, -0.22020508,
        0.13924021,  0.14540714,  0.21312988,  0.08172394,  0.12143619,
       -0.16095696,  0.02420933,  0.11900446, -0.18575825,  0.15674809,
        0.08407095,  0.01523454, -0.03249693, -0.08601876,  0.17967992,
        0.15243264, -0.09245

## **Pre-trained Model**

This model is provided by the dlib library. Dlib's face recognition model is based on classical machine learning techniques rather than deep learning. It uses a combination of HOG (Histogram of Oriented Gradients) features and a SVM (Support Vector Machine) classifier to identify and recognize faces.

### Obtain Users' Face Embeddings

In [ ]:
cap = cv2.VideoCapture(0) # Open a connection to the webcam (0 represents the default camera)

while True:
    ret, frame = cap.read() # Read a frame from the webcam

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert the frame to grayscale for face detection
    
    faces = detector(gray) # Detect faces in the frame

    # Iterate over detected faces
    for face in faces:
        landmarks = face_recognizer(frame, face) # Get facial landmarks

        # Compare with known faces
        for i, known_face in enumerate(known_faces):

            if landmarks == known_face:
                name = known_names[i]
                print(f"Hello, {name}!")

        # Draw a rectangle around the face
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the result
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # Break the loop if the 'q' key is pressed
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
